In [308]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('working-df/av_data-dropped-col-duplicates.csv', encoding='latin-1')

df.info()

C:\Users\Joseph Harvey\AppData\Local\Temp\ipykernel_17444\2880470974.py:6: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('working-df/av_data-dropped-col-duplicates.csv', encoding='latin-1')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87951 entries, 0 to 87950
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event_Id                87951 non-null  object 
 1   Investigation_Type      87951 non-null  object 
 2   Accident_Number         87951 non-null  object 
 3   Event_Date              87951 non-null  object 
 4   Location                87899 non-null  object 
 5   Country                 87729 non-null  object 
 6   Airport_Code            49484 non-null  object 
 7   Airport_Name            52031 non-null  object 
 8   Injury_Severity         86961 non-null  object 
 9   Aircraft_damage         84848 non-null  object 
 10  Aircraft_Category       32181 non-null  object 
 11  Registration_Number     86601 non-null  object 
 12  Make                    87888 non-null  object 
 13  Model                   87859 non-null  object 
 14  Amateur_Built           87851 non-null

Let's look at this warning about mixed types in column 24, Broad_phase_of_flight

In [309]:
# Look at mixed types in Column 24 values including NaN
df.iloc[:,24].apply(type)

0          <class 'str'>
1          <class 'str'>
2          <class 'str'>
3          <class 'str'>
4          <class 'str'>
              ...       
87946    <class 'float'>
87947    <class 'float'>
87948    <class 'float'>
87949    <class 'float'>
87950    <class 'float'>
Name: Broad_phase_of_flight, Length: 87951, dtype: object

In [310]:
df.iloc[:,24].apply(type).value_counts()

Broad_phase_of_flight
<class 'str'>      60837
<class 'float'>    27114
Name: count, dtype: int64

In [311]:
# Show the float values in Column 24
df.iloc[:,24][df.iloc[:,24].apply(type) == float]

2990     NaN
3505     NaN
3591     NaN
3983     NaN
5440     NaN
        ... 
87946    NaN
87947    NaN
87948    NaN
87949    NaN
87950    NaN
Name: Broad_phase_of_flight, Length: 27114, dtype: object

So the float values are referring to NaN values in the column that otherwise consist of string values. Let's fill in the NaN values with the string "Unknown", since we would not be able to identify the true value

In [312]:
# Fill in Broad_phase_of_flight NaN values with 'Unknown'
df['Broad_phase_of_flight'] = df['Broad_phase_of_flight'].fillna('Unknown')

df.iloc[:,24].apply(type).value_counts()

Broad_phase_of_flight
<class 'str'>    87951
Name: count, dtype: int64

In [313]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87951 entries, 0 to 87950
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event_Id                87951 non-null  object 
 1   Investigation_Type      87951 non-null  object 
 2   Accident_Number         87951 non-null  object 
 3   Event_Date              87951 non-null  object 
 4   Location                87899 non-null  object 
 5   Country                 87729 non-null  object 
 6   Airport_Code            49484 non-null  object 
 7   Airport_Name            52031 non-null  object 
 8   Injury_Severity         86961 non-null  object 
 9   Aircraft_damage         84848 non-null  object 
 10  Aircraft_Category       32181 non-null  object 
 11  Registration_Number     86601 non-null  object 
 12  Make                    87888 non-null  object 
 13  Model                   87859 non-null  object 
 14  Amateur_Built           87851 non-null

## Aircraft_Category
The column for Aircraft_Category is also mostly empty, but that data could be useful. The business is after all seeking data related to types of aircraft and airplanes specifically, so removing the column entirely would not work well. Simply removing all rows that do not have a category entry would greatly reduce the number of total rows available for analysis, and most of those removed would likely be airplanes.

I would like to explore the idea of filling in as many of the missing values as I can. This could be done to some extent by making use of the Make column. For example, for all Aircraft_Category entries that are present, if the Make entry is Cessna, then the Aircraft_Category is invariably Airplane. So the missing category entries for Cessnas could reasonably be filled in. The same is true for other Makes like Sikorsky (Helicopter).

See below for an example:

In [314]:
# Aircraft_Category values for Cessna in the Make column
df[df['Make'] == 'Cessna']['Aircraft_Category'].unique()

array([nan, 'Airplane'], dtype=object)

In [315]:
# Show how many nan Aircraft_Category values there are for Cessna
df[df['Make'] == 'Cessna']['Aircraft_Category'].isna().sum()

18344

In [316]:
# Show how many 'Airplane' Aircraft_Category values there are for Cessna
df[(df['Make'] == 'Cessna') & (df['Aircraft_Category'] == 'Airplane')]['Aircraft_Category'].count()

3581

So we can add another 18344 airplane entries to our data by filling in the missing value for Cessna in the Aircraft_Category column

In [317]:
# Fill in Aircraft_Category as 'Airplane' for Cessna
df.loc[df['Make'] == 'Cessna', 'Aircraft_Category'] = 'Airplane'

In [318]:
# Show how many 'Airplane' Aircraft_Category values there are for Cessna
df[(df['Make'] == 'Cessna') & (df['Aircraft_Category'] == 'Airplane')]['Aircraft_Category'].count()

21925

In [319]:
# print a list of the unique values in the Make column
df['Make'].value_counts()

Make
Cessna           21925
Piper            11903
CESSNA            4914
Beech             4290
PIPER             2841
                 ...  
Geertz               1
Conrad Menzel        1
Blucher              1
Gideon               1
ROYSE RALPH L        1
Name: count, Length: 8202, dtype: int64

I realize here that I need to do some further cleaning of the Make column so Cessna and CESSNA (and other similar issues) are not separate values.
How many variants of Cessna are there in the Make column

In [320]:
# Show Make value beginning with Ces, ignoring NaN rows and case
df[df['Make'].str.lower().str.startswith('ces', na=False)].value_counts('Make')

Make
Cessna                     21925
CESSNA                      4914
CESSNA AIRCRAFT CO            24
CESSNA AIRCRAFT                9
CESSNA AIRCRAFT COMPANY        9
Cessna Ector                   3
CESSNA ECTOR                   3
Cessna Aircraft Company        3
Cessna Wren                    2
CESSNA/AIR REPAIR INC          2
CESSNA/WEAVER                  1
Cessna Aircraft Co.            1
CESSNA REIMS                   1
CESSNA Aircraft                1
Cessna Reems                   1
Cessna Robertson               1
Cessna Skyhawk II              1
Cessna Soloy                   1
Cesna                          1
Name: count, dtype: int64

In [321]:
# Convert all these cessna values to 'Cessna'
df.loc[df['Make'].str.lower().str.startswith('ces', na=False), 'Make'] = 'Cessna'

In [322]:
# Show Make value beginning with Ces, ignoring NaN rows and case
df[df['Make'].str.lower().str.startswith('ces', na=False)].value_counts('Make')

Make
Cessna    26903
Name: count, dtype: int64

So now we have almost 27000 Cessna makes in the Make column. Let's look at the category again

In [323]:
# Aircraft_Category values for Cessna in the Make column, include NaN
df[df['Make'] == 'Cessna'].value_counts('Aircraft_Category', dropna=False)

Aircraft_Category
Airplane    26840
NaN            62
Unknown         1
Name: count, dtype: int64

In [324]:
# Fill in Aircraft_Category as 'Airplane' for Cessna
df.loc[df['Make'] == 'Cessna', 'Aircraft_Category'] = 'Airplane'

In [325]:
# Show how many 'Airplane' Aircraft_Category values there are for Cessna now
df[(df['Make'] == 'Cessna') & (df['Aircraft_Category'] == 'Airplane')]['Aircraft_Category'].count()

26903

I need to do the same for other Makes.

In [326]:
# Show most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('a', na=False)].value_counts('Make').head(20)

Make
Air Tractor                   588
Aeronca                       479
Aero Commander                356
Aerospatiale                  334
AIRBUS                        251
AIR TRACTOR INC               217
Ayres                         213
AERONCA                       149
Airbus Industrie              135
Aviat                         112
AIR TRACTOR                    96
Aerostar                       85
AVIAT AIRCRAFT INC             72
AERO COMMANDER                 69
American                       58
AEROSPATIALE                   46
AMERICAN CHAMPION AIRCRAFT     46
Agusta                         44
AYRES CORPORATION              38
Aerotek                        38
Name: count, dtype: int64

I see some duplicates here through different cases, etc. These can be cleaned up by assigning a consistent spelling

In [327]:
df.loc[df['Make'].str.lower().str.startswith('aero comm', na=False), 'Make'] = 'Aero Commander'

In [328]:
df.loc[df['Make'].str.lower().str.startswith('air tra', na=False), 'Make'] = 'Air Tractor'

In [329]:
df.loc[df['Make'].str.lower().str.startswith('aeron', na=False), 'Make'] = 'Aeronca'

In [330]:
df.loc[df['Make'].str.lower().str.startswith('ayre', na=False), 'Make'] = 'Ayres'

In [331]:
df.loc[df['Make'].str.lower().str.startswith('aerosp', na=False), 'Make'] = 'Aerospatiale'

In [332]:
# Show most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('a', na=False)].value_counts('Make').head(20)

Make
Air Tractor                   911
Aeronca                       633
Aero Commander                425
Aerospatiale                  395
Ayres                         284
AIRBUS                        251
Airbus Industrie              135
Aviat                         112
Aerostar                       85
AVIAT AIRCRAFT INC             72
American                       58
AMERICAN CHAMPION AIRCRAFT     46
Agusta                         44
Aerotek                        38
Airbus                         37
Alon                           30
AVIAT                          28
American Champion (acac)       28
Adams                          23
AIRBUS INDUSTRIE               22
Name: count, dtype: int64

In [333]:
df.loc[df['Make'].str.lower().str.startswith('airb', na=False), 'Make'] = 'Airbus'

In [334]:
df.loc[df['Make'].str.lower().str.startswith('avia', na=False), 'Make'] = 'Aviat'

In [335]:
# Show most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('a', na=False)].value_counts('Make').head(20)

Make
Air Tractor                      911
Aeronca                          633
Airbus                           506
Aero Commander                   425
Aerospatiale                     395
Ayres                            284
Aviat                            259
Aerostar                          85
American                          58
AMERICAN CHAMPION AIRCRAFT        46
Agusta                            44
Aerotek                           38
Alon                              30
American Champion (acac)          28
Adams                             23
AGUSTA                            21
American Aviation Corp. (aac)     20
ATR                               19
Artic Aircraft Corp.              17
Air Command                       17
Name: count, dtype: int64

In [336]:
df.loc[df['Make'].str.lower().str.startswith('agus', na=False), 'Make'] = 'Agusta'

In [337]:
# Show most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('a', na=False)].value_counts('Make').head(20)

Make
Air Tractor                      911
Aeronca                          633
Airbus                           506
Aero Commander                   425
Aerospatiale                     395
Ayres                            284
Aviat                            259
Agusta                            92
Aerostar                          85
American                          58
AMERICAN CHAMPION AIRCRAFT        46
Aerotek                           38
Alon                              30
American Champion (acac)          28
Adams                             23
American Aviation Corp. (aac)     20
ATR                               19
Air Command                       17
Artic Aircraft Corp.              17
AMERICAN LEGEND AIRCRAFT CO       17
Name: count, dtype: int64

In [338]:
df.loc[df['Make'].str.lower().str.startswith('american cha', na=False), 'Make'] = 'American Champion'

In [339]:
df.loc[df['Make'].str.lower().str.startswith('american av', na=False), 'Make'] = 'American'

In [340]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american l', na=False)].value_counts('Make').head(20)

Make
AMERICAN LEGEND AIRCRAFT CO       17
American Legend                    7
AMERICAN LEGEND                    2
American Legend Aircraft Co.       2
AMERICAN LONGEVITY CORP            1
American Legand Aircraft           1
American Legend Aircraft Compa     1
Name: count, dtype: int64

In [341]:
df.loc[df['Make'].str.lower().str.startswith('american leg', na=False), 'Make'] = 'American Legend'

In [342]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american g', na=False)].value_counts('Make').head(20)

Make
American General Aircraft     16
AMERICAN GENERAL ACFT CORP     3
Name: count, dtype: int64

In [343]:
df.loc[df['Make'].str.lower().str.startswith('american g', na=False), 'Make'] = 'American General Aircraft'

In [344]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american', na=False)].value_counts('Make').head(20)

Make
American                      107
American Champion             102
American Legend                30
American General Aircraft      19
AMERICAN                       14
American Aerolights            10
AMERICAN EUROCOPTER CORP        6
American Eurocopter             6
American Blimp Corp.            5
AMERICAN EUROCOPTER LLC         4
AMERICAN EUROCOPTER             2
American Eagle                  2
American Autogyro               2
American Aircraft               1
American Blimp Corporation      1
AMERICAN AIR RACING LTD         1
American Air Racing             1
AMERICAN LONGEVITY CORP         1
AMERICAN BLIMP                  1
American Yankee                 1
Name: count, dtype: int64

In [345]:
# Change AMERICAN to American
df.loc[(df['Make'] == 'American') | (df['Make'] == 'AMERICAN'), 'Make'] = 'American'

In [346]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american', na=False)].value_counts('Make').head(20)

Make
American                      121
American Champion             102
American Legend                30
American General Aircraft      19
American Aerolights            10
AMERICAN EUROCOPTER CORP        6
American Eurocopter             6
American Blimp Corp.            5
AMERICAN EUROCOPTER LLC         4
AMERICAN EUROCOPTER             2
American Autogyro               2
American Eagle                  2
AMERICAN AIR RACING LTD         1
American Aircraft               1
American Blimp Corporation      1
AMERICAN BLIMP                  1
American Air Racing             1
AMERICAN LONGEVITY CORP         1
American Yankee                 1
Name: count, dtype: int64

In [347]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american b', na=False)].value_counts('Make').head(20)

Make
American Blimp Corp.          5
AMERICAN BLIMP                1
American Blimp Corporation    1
Name: count, dtype: int64

In [348]:
df.loc[df['Make'].str.lower().str.startswith('american b', na=False), 'Make'] = 'American Blimp'

In [349]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american', na=False)].value_counts('Make').head(20)

Make
American                     121
American Champion            102
American Legend               30
American General Aircraft     19
American Aerolights           10
American Blimp                 7
American Eurocopter            6
AMERICAN EUROCOPTER CORP       6
AMERICAN EUROCOPTER LLC        4
AMERICAN EUROCOPTER            2
American Autogyro              2
American Eagle                 2
AMERICAN AIR RACING LTD        1
American Aircraft              1
American Air Racing            1
AMERICAN LONGEVITY CORP        1
American Yankee                1
Name: count, dtype: int64

In [350]:
# Show me most popular makes beginning with 'A'
df[df['Make'].str.lower().str.startswith('american eu', na=False)].value_counts('Make').head(20)

Make
AMERICAN EUROCOPTER CORP    6
American Eurocopter         6
AMERICAN EUROCOPTER LLC     4
AMERICAN EUROCOPTER         2
Name: count, dtype: int64

In [351]:
df.loc[df['Make'].str.lower().str.startswith('american eu', na=False), 'Make'] = 'American Eurocopter'

In [352]:
# Show me most popular makes beginning with 'B'
df[df['Make'].str.lower().str.startswith('b', na=False)].value_counts('Make').head(20)

Make
Beech                             4290
Bell                              2118
Boeing                            1512
BOEING                            1140
BEECH                             1042
Bellanca                           874
BELL                               588
BELLANCA                           159
Balloon Works                      135
Burkhart Grob                      109
British Aerospace                   84
BOMBARDIER INC                      68
Boeing Stearman                     48
BOMBARDIER                          46
Brantly Helicopter                  39
Britten-norman                      37
Bombardier                          30
Beechcraft                          24
BELL HELICOPTER TEXTRON CANADA      23
Barnes                              22
Name: count, dtype: int64

In [353]:
df.loc[df['Make'].str.lower().str.startswith('beec', na=False), 'Make'] = 'Beech'

In [354]:
# Show me most popular makes beginning with 'B'
df[df['Make'].str.lower().str.startswith('bell', na=False)].value_counts('Make').head(20)

Make
Bell                              2118
Bellanca                           874
BELL                               588
BELLANCA                           159
BELL HELICOPTER TEXTRON CANADA      23
BELL HELICOPTER TEXTRON             21
BELL HELICOPTER                      4
Bell-transworld                      3
Bell Helicopter                      3
Bell-k Copter                        2
Bell Helicopter Textron              2
Bell-carson                          2
BELL TEXTRON CANADA LTD              2
BELL HELICOPTER CO                   2
Bell-olympic Helicopters, Inc.       1
Bell-moore                           1
Bell-world                           1
Bell/soloy                           1
Bell/garlick                         1
Bell/mason                           1
Name: count, dtype: int64

In [355]:
df.loc[df['Make'].str.lower().str.startswith('bell h', na=False), 'Make'] = 'Bell'

In [356]:
df[df['Make'].str.lower().str.startswith('bell', na=False)].value_counts('Make').head(20)

Make
Bell                              2174
Bellanca                           874
BELL                               588
BELLANCA                           159
Bell-transworld                      3
Bell-carson                          2
Bell-k Copter                        2
BELL TEXTRON CANADA LTD              2
Bell/tsirah                          1
Bell/textron                         1
Bellah                               1
Bell-kitz Kopters                    1
Bell/mason                           1
Bellanca Aircraft Corporation        1
Bell/garlick                         1
Bell-world                           1
Bellanca Citabria                    1
Bell-olympic Helicopters, Inc.       1
Bell-moore                           1
Bell/soloy                           1
Name: count, dtype: int64

In [357]:
df.loc[df['Make'].str.lower().str.startswith('bella', na=False), 'Make'] = 'Bellanca'

In [358]:
df[df['Make'].str.lower().str.startswith('bell', na=False)].value_counts('Make').head(20)

Make
Bell                              2174
Bellanca                          1037
BELL                               588
Bell-transworld                      3
Bell-carson                          2
BELL TEXTRON CANADA LTD              2
Bell-k Copter                        2
Bell-continental Copters             1
Bell/tsirah                          1
Bell/textron                         1
Bell/soloy                           1
Bell/mason                           1
Bell/garlick                         1
Bell-world                           1
Bell-olympic Helicopters, Inc.       1
Bell-moore                           1
Bell-kitz Kopters                    1
BELL-CARSON                          1
BELL-TELLIJOHN                       1
BELL BILL                            1
Name: count, dtype: int64

In [359]:
# Change BELL to Bell
df.loc[(df['Make'] == 'Bell') | (df['Make'] == 'BELL'), 'Make'] = 'Bell'

In [360]:
df[df['Make'].str.lower().str.startswith('bell', na=False)].value_counts('Make').head(20)

Make
Bell                              2762
Bellanca                          1037
Bell-transworld                      3
Bell-k Copter                        2
BELL TEXTRON CANADA LTD              2
Bell-carson                          2
BELL-WATSON                          1
Bell-kitz Kopters                    1
Bell/tsirah                          1
Bell/textron                         1
Bell/soloy                           1
Bell/mason                           1
Bell/garlick                         1
Bell-world                           1
BELL-CARSON                          1
Bell-olympic Helicopters, Inc.       1
Bell-moore                           1
BELL-TELLIJOHN                       1
BELL/GARLICK                         1
Bell-continental Copters             1
Name: count, dtype: int64

In [361]:
df.loc[df['Make'].str.lower().str.startswith('bell-', na=False), 'Make'] = 'Bell'

In [362]:
df.loc[df['Make'].str.lower().str.startswith('bell/', na=False), 'Make'] = 'Bell'

In [363]:
df[df['Make'].str.lower().str.startswith('bell', na=False)].value_counts('Make').head(20)

Make
Bell                       2787
Bellanca                   1037
BELL TEXTRON CANADA LTD       2
BELL BILL                     1
BELLER                        1
BELLET JAMES J                1
Bell 47g-3b                   1
Bell Shelby Aero              1
Name: count, dtype: int64

In [364]:
df.loc[df['Make'].str.lower().str.startswith('bell t', na=False), 'Make'] = 'Bell'

In [365]:
df.loc[df['Make'].str.lower().str.startswith('bell b', na=False), 'Make'] = 'Bell'

In [366]:
df.loc[df['Make'].str.lower().str.startswith('bell 4', na=False), 'Make'] = 'Bell'

In [367]:
df.loc[df['Make'].str.lower().str.startswith('bell s', na=False), 'Make'] = 'Bell'

In [368]:
df[df['Make'].str.lower().str.startswith('bell', na=False)].value_counts('Make').head(20)

Make
Bell              2792
Bellanca          1037
BELLER               1
BELLET JAMES J       1
Name: count, dtype: int64

In [369]:
# Show me most popular makes beginning with 'B'
df[df['Make'].str.lower().str.startswith('boe', na=False)].value_counts('Make').head(20)

Make
Boeing                            1512
BOEING                            1140
Boeing Stearman                     48
BOEING COMPANY                       8
Boeing Vertol                        6
Boeing Helicopters Div.              3
Boeing - Canada (de Havilland)       2
Boeing (Stearman)                    1
Boeing Company                       1
Boeing Commercial Airplane Gro       1
BOEHLKE KEVIN P                      1
BOEVE EARL                           1
BOEING-VERTOL                        1
BOEING-STEARMAN                      1
BOEING OF CANADA/DEHAV DIV           1
BOEING COMPANY, LONG BEACH DIV       1
BOEING 777-306ER                     1
Boeing-brown                         1
Name: count, dtype: int64

In [370]:
df.loc[df['Make'].str.lower().str.startswith('boeing h', na=False), 'Make'] = 'Boeing Helicopters'

In [371]:
df.loc[df['Make'].str.lower().str.startswith('boeing c', na=False), 'Make'] = 'Boeing'

In [372]:
df[df['Make'].str.lower().str.startswith('boe', na=False)].value_counts('Make').head(20)

Make
Boeing                            1523
BOEING                            1140
Boeing Stearman                     48
Boeing Vertol                        6
Boeing Helicopters                   3
Boeing - Canada (de Havilland)       2
BOEHLKE KEVIN P                      1
BOEING 777-306ER                     1
BOEING OF CANADA/DEHAV DIV           1
BOEING-STEARMAN                      1
BOEING-VERTOL                        1
BOEVE EARL                           1
Boeing (Stearman)                    1
Boeing-brown                         1
Name: count, dtype: int64

In [373]:
df.loc[df['Make'].str.lower().str.startswith('boeing s', na=False), 'Make'] = 'Boeing'

In [374]:
df.loc[df['Make'].str.lower().str.startswith('boeing v', na=False), 'Make'] = 'Boeing'

In [375]:
df.loc[df['Make'].str.lower().str.startswith('boeing - c', na=False), 'Make'] = 'Boeing'

In [376]:
df[df['Make'].str.lower().str.startswith('boe', na=False)].value_counts('Make').head(20)

Make
Boeing                        1579
BOEING                        1140
Boeing Helicopters               3
BOEHLKE KEVIN P                  1
BOEING 777-306ER                 1
BOEING OF CANADA/DEHAV DIV       1
BOEING-STEARMAN                  1
BOEING-VERTOL                    1
BOEVE EARL                       1
Boeing (Stearman)                1
Boeing-brown                     1
Name: count, dtype: int64

In [377]:
# Change BOEING to Boeing
df.loc[(df['Make'] == 'Boeing') | (df['Make'] == 'BOEING'), 'Make'] = 'Boeing'

In [378]:
df[df['Make'].str.lower().str.startswith('b', na=False)].value_counts('Make').head(20)

Make
Beech                 5372
Bell                  2792
Boeing                2719
Bellanca              1037
Balloon Works          135
Burkhart Grob          109
British Aerospace       84
BOMBARDIER INC          68
BOMBARDIER              46
Brantly Helicopter      39
Britten-norman          37
Bombardier              30
Barnes                  22
Bombardier, Inc.        22
BAE                     21
Benson                  18
Bensen                  17
BRANTLY                 13
Bede Aircraft           12
BRITTEN NORMAN          11
Name: count, dtype: int64

In [379]:
df.loc[df['Make'].str.lower().str.startswith('bombard', na=False), 'Make'] = 'Bombardier'

In [380]:
df.loc[df['Make'].str.lower().str.startswith('bens', na=False), 'Make'] = 'Benson'

In [381]:
df[df['Make'].str.lower().str.startswith('b', na=False)].value_counts('Make').head(20)

Make
Beech                 5372
Bell                  2792
Boeing                2719
Bellanca              1037
Bombardier             169
Balloon Works          135
Burkhart Grob          109
British Aerospace       84
Benson                  39
Brantly Helicopter      39
Britten-norman          37
Barnes                  22
BAE                     21
BRANTLY                 13
Bede Aircraft           12
BURKHART GROB           11
Blanik                  11
BRITTEN NORMAN          11
Bushby                  11
Buckeye                 10
Name: count, dtype: int64

In [382]:
df[df['Make'].str.lower().str.startswith('c', na=False)].value_counts('Make').head(20)

Make
Cessna                            26903
Champion                            426
CIRRUS DESIGN CORP                  218
CHAMPION                             91
CIRRUS                               80
Cirrus Design Corp.                  71
Cirrus                               70
Canadair                             62
Cameron                              54
Christen Industries                  53
Convair                              39
COSTRUZIONI AERONAUTICHE TECNA       31
Continental Copters                  28
Callair                              26
Consolidated Aeronautics Inc.        25
Cameron Balloons                     21
Casa                                 20
CUBCRAFTERS INC                      19
CUB CRAFTERS INC                     19
Classic Aircraft Corp.               17
Name: count, dtype: int64

In [383]:
df.loc[df['Make'].str.lower().str.startswith('champ', na=False), 'Make'] = 'Champion'

In [384]:
df.loc[df['Make'].str.lower().str.startswith('cirrus', na=False), 'Make'] = 'Cirrus Design'

In [385]:
df.loc[df['Make'].str.lower().str.startswith('cub', na=False), 'Make'] = 'Cubcrafters'

In [386]:
df[df['Make'].str.lower().str.startswith('c', na=False)].value_counts('Make').head(20)

Make
Cessna                            26903
Champion                            517
Cirrus Design                       467
Cubcrafters                          81
Canadair                             62
Cameron                              54
Christen Industries                  53
Convair                              39
COSTRUZIONI AERONAUTICHE TECNA       31
Continental Copters                  28
Callair                              26
Consolidated Aeronautics Inc.        25
Cameron Balloons                     21
Casa                                 20
Curtiss-wright                       17
Classic Aircraft Corp.               17
CHRISTEN INDUSTRIES INC              16
Consolidated-vultee                  15
Culver                               15
COLUMBIA AIRCRAFT MFG                13
Name: count, dtype: int64

In [387]:
df.loc[df['Make'].str.lower().str.startswith('christ', na=False), 'Make'] = 'Christen Industries'

In [388]:
df[df['Make'].str.lower().str.startswith('c', na=False)].value_counts('Make').head(20)

Make
Cessna                            26903
Champion                            517
Cirrus Design                       467
Christen Industries                  93
Cubcrafters                          81
Canadair                             62
Cameron                              54
Convair                              39
COSTRUZIONI AERONAUTICHE TECNA       31
Continental Copters                  28
Callair                              26
Consolidated Aeronautics Inc.        25
Cameron Balloons                     21
Casa                                 20
Curtiss-wright                       17
Classic Aircraft Corp.               17
Culver                               15
Consolidated-vultee                  15
CONSOLIDATED AERONAUTICS INC.        13
COLUMBIA AIRCRAFT MFG                13
Name: count, dtype: int64

In [389]:
df.loc[df['Make'].str.lower().str.startswith('consol', na=False), 'Make'] = 'Consolidated Aeronautics'

In [390]:
df[df['Make'].str.lower().str.startswith('c', na=False)].value_counts('Make').head(20)

Make
Cessna                            26903
Champion                            517
Cirrus Design                       467
Christen Industries                  93
Cubcrafters                          81
Consolidated Aeronautics             63
Canadair                             62
Cameron                              54
Convair                              39
COSTRUZIONI AERONAUTICHE TECNA       31
Continental Copters                  28
Callair                              26
Cameron Balloons                     21
Casa                                 20
Classic Aircraft Corp.               17
Curtiss-wright                       17
Culver                               15
COLUMBIA AIRCRAFT MFG                13
CAMERON BALLOONS US                  12
CZECH AIRCRAFT WORKS SPOL SRO        12
Name: count, dtype: int64

In [391]:
# Show category value_counts for Cameron
df[df['Make'] == 'Cameron'].value_counts('Aircraft_Category', dropna=False)

Aircraft_Category
NaN        47
Balloon     7
Name: count, dtype: int64

In [392]:
df.loc[df['Make'].str.lower().str.startswith('camer', na=False), 'Make'] = 'Cameron Balloons'

In [393]:
df[df['Make'].str.lower().str.startswith('c', na=False)].value_counts('Make').head(20)

Make
Cessna                            26903
Champion                            517
Cirrus Design                       467
Cameron Balloons                    105
Christen Industries                  93
Cubcrafters                          81
Consolidated Aeronautics             63
Canadair                             62
Convair                              39
COSTRUZIONI AERONAUTICHE TECNA       31
Continental Copters                  28
Callair                              26
Casa                                 20
Classic Aircraft Corp.               17
Curtiss-wright                       17
Culver                               15
COLUMBIA AIRCRAFT MFG                13
CZECH AIRCRAFT WORKS SPOL SRO        12
Commander                            12
CZECH SPORT AIRCRAFT AS              11
Name: count, dtype: int64

In [394]:
df[df['Make'].str.lower().str.startswith('d', na=False)].value_counts('Make').head(20)

Make
De Havilland                   370
Douglas                        250
DEHAVILLAND                     91
DIAMOND AIRCRAFT IND INC        74
Diamond Aircraft Industries     42
DE HAVILLAND                    31
DIAMOND                         30
DOUGLAS                         26
Dornier                         25
Dassault                        22
Diamond                         19
Davis                           19
Dassault-breguet                16
Dassault Aviation               11
de Havilland                     9
Dehavilland                      8
DIAMOND AIRCRAFT IND GMBH        8
DORNIER                          7
DASSAULT                         7
Denney                           6
Name: count, dtype: int64

In [395]:
df.loc[df['Make'].str.lower().str.startswith('de h', na=False), 'Make'] = 'De Havilland'

In [396]:
df.loc[df['Make'].str.lower().str.startswith('dehav', na=False), 'Make'] = 'De Havilland'

In [397]:
df.loc[df['Make'].str.lower().str.startswith('dassault', na=False), 'Make'] = 'Dassault'

In [398]:
df[df['Make'].str.lower().str.startswith('d', na=False)].value_counts('Make').head(20)

Make
De Havilland                   512
Douglas                        250
DIAMOND AIRCRAFT IND INC        74
Dassault                        72
Diamond Aircraft Industries     42
DIAMOND                         30
DOUGLAS                         26
Dornier                         25
Davis                           19
Diamond                         19
DIAMOND AIRCRAFT IND GMBH        8
DORNIER                          7
Denney                           6
DESTINY                          5
Downer Aircraft Industries       4
Diamond Aircraft                 4
DG FLUGZEUGBAU GMBH              4
Downer                           3
Day                              3
Drake                            3
Name: count, dtype: int64

In [399]:
df.loc[df['Make'].str.lower().str.startswith('diam', na=False), 'Make'] = 'Diamond'

In [400]:
df[df['Make'].str.lower().str.startswith('d', na=False)].value_counts('Make').head(20)

Make
De Havilland                  512
Douglas                       250
Diamond                       182
Dassault                       72
DOUGLAS                        26
Dornier                        25
Davis                          19
DORNIER                         7
Denney                          6
DESTINY                         5
Downer Aircraft Industries      4
DG FLUGZEUGBAU GMBH             4
Davenport                       3
Downer                          3
DJI                             3
Day                             3
Drake                           3
DAVIS MICHAEL J                 2
DAY WILLIAM L                   2
DAYTON A BABCOCK                2
Name: count, dtype: int64

In [401]:
df.loc[df['Make'].str.lower().str.startswith('doug', na=False), 'Make'] = 'Douglas'

In [402]:
df.loc[df['Make'].str.lower().str.startswith('dorn', na=False), 'Make'] = 'Dornier'

In [403]:
df[df['Make'].str.lower().str.startswith('e', na=False)].value_counts('Make').head(30)

Make
Enstrom                           247
Ercoupe (eng & Research Corp.)    155
EMBRAER                           130
EUROCOPTER                        128
Embraer                           106
Eurocopter                         90
Ercoupe                            54
ENSTROM                            49
Eipper                             43
Eagle Aircraft Co.                 42
Eurocopter France                  40
ERCOUPE                            34
EVEKTOR-AEROTECHNIK AS             26
EUROCOPTER DEUTSCHLAND GMBH        20
Extra Flugzeugbau                  19
Eiriavion Oy                       17
ENGINEERING & RESEARCH             15
Erco                               11
EXTRA FLUGZEUGBAU GMBH             10
Evans Aircraft                     10
Eagle                              10
Eurocopter Deutschland             10
EMBRAER S A                         9
Experimental                        9
EVOLUTION AIRCRAFT INC              9
Extra                               7
ENSTROM

In [404]:
df.loc[df['Make'].str.lower().str.startswith('eagl', na=False), 'Make'] = 'Eagle Aircraft'

In [405]:
df.loc[df['Make'].str.lower().str.startswith('embr', na=False), 'Make'] = 'Embraer'

In [406]:
df.loc[df['Make'].str.lower().str.startswith('enstrom', na=False), 'Make'] = 'Enstrom'

In [407]:
df.loc[df['Make'].str.lower().str.startswith('ercou', na=False), 'Make'] = 'Ercoupe'

In [408]:
df.loc[df['Make'].str.lower().str.startswith('euroc', na=False), 'Make'] = 'Eurocopter'

In [409]:
df.loc[df['Make'].str.lower().str.startswith('evek', na=False), 'Make'] = 'Evektor Aerotechnik'

In [410]:
df.loc[df['Make'].str.lower().str.startswith('extra', na=False), 'Make'] = 'Extra'

In [411]:
df[df['Make'].str.lower().str.startswith('e', na=False)].value_counts('Make').head(30)

Make
Enstrom                         303
Eurocopter                      295
Embraer                         257
Ercoupe                         247
Eagle Aircraft                   65
Extra                            52
Evektor Aerotechnik              44
Eipper                           43
Eiriavion Oy                     17
ENGINEERING & RESEARCH           15
Erco                             11
Evans Aircraft                   10
EVOLUTION AIRCRAFT INC            9
Experimental                      9
ECLIPSE AVIATION CORP             5
EVOLUTION TRIKES                  5
Emair                             5
Engineering & Research            5
Evolution                         4
Erickson                          4
Elliott                           4
Eaa Biplane                       4
Ellis                             3
Eaa                               3
EIPPER                            3
Emroth Emair                      3
Eclipse Aviation Corporation      3
EIRIAVION OY           

In [412]:
df.loc[df['Make'].str.lower().str.startswith('eaa', na=False), 'Make'] = 'Eaa'

In [413]:
df.loc[df['Make'].str.lower().str.startswith('ecli', na=False), 'Make'] = 'Eclipse Aviation'

In [414]:
df.loc[df['Make'].str.lower().str.startswith('eip', na=False), 'Make'] = 'Eipper'

In [415]:
df.loc[df['Make'].str.lower().str.startswith('eiri', na=False), 'Make'] = 'Eiriavion Oy'

In [416]:
df.loc[df['Make'].str.lower().str.startswith('eng', na=False), 'Make'] = 'Engineering & Research'

In [417]:
df.loc[df['Make'].str.lower().str.startswith('evol', na=False), 'Make'] = 'Evolution'

In [418]:
df[df['Make'].str.lower().str.startswith('f', na=False)].value_counts('Make').head(30)

Make
Fairchild                  131
Fokker                      56
FLIGHT DESIGN GMBH          42
Fairchild Hiller            35
FAIRCHILD                   27
Flight Design               20
Forney                      17
Fairchild Swearingen        11
Fleet                       11
FLIGHT DESIGN                9
FOKKER                       8
Fisher                       7
FLEET                        6
FANTASY AIR SRO              5
Firefly                      5
Fantasy Sky Promotions       4
FISHER                       4
Flight Design Gmbh           4
FOUND ACFT CANADA INC        4
Flagg                        4
Funk, D.d. Aviation Co.      4
Firefly Balloons             4
FAIRCHILD HILLER             4
Flight Design GMBH           3
FLIGHTSTAR                   3
Fouga                        3
Fetherolf                    3
FLIGHTSTAR INC               3
Fairchild Dornier            3
Fischer                      3
Name: count, dtype: int64

In [419]:
df.loc[df['Make'].str.lower().str.startswith('fairch', na=False), 'Make'] = 'Fairchild'

In [420]:
df.loc[df['Make'].str.lower().str.startswith('firef', na=False), 'Make'] = 'Firefly'

In [421]:
df.loc[df['Make'].str.lower().str.startswith('fish', na=False), 'Make'] = 'Fisher'

In [422]:
df.loc[df['Make'].str.lower().str.startswith('fleet', na=False), 'Make'] = 'Fleet'

In [423]:
df.loc[df['Make'].str.lower().str.startswith('flight d', na=False), 'Make'] = 'Flight Design'

In [424]:
df.loc[df['Make'].str.lower().str.startswith('flights', na=False), 'Make'] = 'Flightstar'

In [425]:
df.loc[df['Make'].str.lower().str.startswith('fokk', na=False), 'Make'] = 'Fokker'

In [426]:
df[df['Make'].str.lower().str.startswith('f', na=False)].value_counts('Make').head(30)

Make
Fairchild                    221
Flight Design                 81
Fokker                        64
Fisher                        21
Fleet                         18
Forney                        17
Firefly                       15
Flightstar                    10
FANTASY AIR SRO                5
Funk, D.d. Aviation Co.        4
Fantasy Sky Promotions         4
FOUND ACFT CANADA INC          4
Flagg                          4
Fetherolf                      3
FPNA LLC                       3
Ferguson                       3
Fouga                          3
Fischer                        3
Franklin                       3
Farrington                     2
FOUGA                          2
Focke-wulf                     2
Fagan                          2
Fantasy Air                    2
Freeman                        2
Folland                        2
Fowler                         2
Found Aircraft Canada Inc      2
Flugzeugbau                    2
FUNK                           2
Name:

In [427]:
df.loc[df['Make'].str.lower().str.startswith('foug', na=False), 'Make'] = 'Fouga'

In [428]:
df.loc[df['Make'].str.lower().str.startswith('found', na=False), 'Make'] = 'Found Aircraft'

In [429]:
df.loc[df['Make'].str.lower().str.startswith('funk', na=False), 'Make'] = 'Funk'

In [430]:
df[df['Make'].str.lower().str.startswith('f', na=False)].value_counts('Make').head(30)

Make
Fairchild                       221
Flight Design                    81
Fokker                           64
Fisher                           21
Fleet                            18
Forney                           17
Firefly                          15
Flightstar                       10
Funk                              9
Found Aircraft                    8
Fouga                             5
FANTASY AIR SRO                   5
Fantasy Sky Promotions            4
Flagg                             4
Ferguson                          3
FPNA LLC                          3
Franklin                          3
Fischer                           3
Fetherolf                         3
Flugzeugbau                       2
Flug-und Fahrzeugwerke (ffa)      2
Fantasy Air                       2
FANTASY AIR                       2
Farrington                        2
Fuji                              2
Folland                           2
Fowler                            2
Fields                 

In [431]:
df.loc[df['Make'].str.lower().str.startswith('fant', na=False), 'Make'] = 'Fantasy'

In [432]:
df[df['Make'].str.lower().str.startswith('g', na=False)].value_counts('Make').head(30)

Make
Grumman                           1080
Grumman American                   222
Grumman-schweizer                  121
Globe                               79
GRUMMAN                             78
Gates Learjet                       63
Gulfstream                          62
GRUMMAN ACFT ENG COR-SCHWEIZER      58
Gulfstream American                 55
Great Lakes                         51
GRUMMAN AMERICAN AVN. CORP.         49
Glasflugel                          25
GULFSTREAM                          24
Garlick                             22
Gulfstream-schweizer                22
Glasair                             20
Gulfstream Aerospace                17
GLOBE                               17
Glaser-dirks                        17
GREAT LAKES                         14
GULFSTREAM AEROSPACE                12
GARLICK                             10
Grob                                 9
Gardner                              9
GULFSTREAM AMERICAN CORP             9
GULFSTREAM-SCHWEIZER

In [433]:
df.loc[df['Make'].str.lower().str.startswith('garl', na=False), 'Make'] = 'Garlick'

In [434]:
df[df['Make'].str.lower().str.startswith('g', na=False)].value_counts('Make').head(50)

Make
Grumman                           1080
Grumman American                   222
Grumman-schweizer                  121
Globe                               79
GRUMMAN                             78
Gates Learjet                       63
Gulfstream                          62
GRUMMAN ACFT ENG COR-SCHWEIZER      58
Gulfstream American                 55
Great Lakes                         51
GRUMMAN AMERICAN AVN. CORP.         49
Garlick                             45
Glasflugel                          25
GULFSTREAM                          24
Gulfstream-schweizer                22
Glasair                             20
Gulfstream Aerospace                17
GLOBE                               17
Glaser-dirks                        17
GREAT LAKES                         14
GULFSTREAM AEROSPACE                12
Grob                                 9
GULFSTREAM-SCHWEIZER A/C CORP        9
Gardner                              9
GULFSTREAM AMERICAN CORP             9
GATES LEARJET CORP. 

In [435]:
df.loc[df['Make'].str.lower().str.startswith('gates', na=False), 'Make'] = 'Gates Learjet'

In [436]:
df.loc[df['Make'].str.lower().str.startswith('general atom', na=False), 'Make'] = 'General Atomics'

In [437]:
df.loc[df['Make'].str.lower().str.startswith('glasa', na=False), 'Make'] = 'Glasair'

In [438]:
df.loc[df['Make'].str.lower().str.startswith('glassa', na=False), 'Make'] = 'Glasair'

In [439]:
df.loc[df['Make'].str.lower().str.startswith('glase', na=False), 'Make'] = 'Glaser Dirks'

In [440]:
df.loc[df['Make'].str.lower().str.startswith('glasf', na=False), 'Make'] = 'Glasflugel'

In [441]:
df.loc[df['Make'].str.lower().str.startswith('globe', na=False), 'Make'] = 'Globe'

In [442]:
df.loc[df['Make'].str.lower().str.startswith('great l', na=False), 'Make'] = 'Great Lakes'

In [443]:
df.loc[df['Make'].str.lower().str.startswith('grob', na=False), 'Make'] = 'Grob'

In [444]:
df.loc[df['Make'].str.lower().str.startswith('grum', na=False), 'Make'] = 'Grumman'

In [445]:
df.loc[df['Make'].str.lower().str.startswith('gulfstr', na=False), 'Make'] = 'Gulfstream'

In [446]:
df[df['Make'].str.lower().str.startswith('g', na=False)].value_counts('Make').head(50)

Make
Grumman                           1632
Gulfstream                         226
Globe                               97
Gates Learjet                       82
Great Lakes                         67
Garlick                             45
Glasair                             34
Glasflugel                          29
Glaser Dirks                        24
Grob                                15
Gardner                              9
General Atomics                      8
Gray                                 7
Government Aircraft Fact (gaf)       6
Guimbal                              6
General Balloon                      6
Green                                4
Griffin                              4
Gordon                               4
Goodyear                             4
Giles                                3
Garrett                              3
Graham                               3
Garner                               2
GOLDEN CIRCLE AIR                    2
Glover              

In [447]:
df.loc[df['Make'].str.lower().str.startswith('golden c', na=False), 'Make'] = 'Golden Circle Air'

In [448]:
df.loc[df['Make'].str.lower().str.startswith('gren', na=False), 'Make'] = 'Grenier'

In [449]:
df[df['Make'].str.lower().str.startswith('h', na=False)].value_counts('Make').head(50)

Make
Hughes                           794
Hiller                           311
HUGHES                           137
Helio                             94
HILLER                            37
Homebuilt                         30
Hawker Siddeley                   28
HELIO                             19
Howard Aircraft Corp.             14
HAWKER BEECHCRAFT CORP            13
Hispano Aviacion                  11
Head Balloons, Inc.               10
Hall                              10
Hiller-soloy                       9
Hawker Beechcraft                  8
HAWKER BEECHCRAFT                  7
Hawker Beechcraft Corp.            7
HAWKER                             7
Hunter                             7
Harris                             6
Hayes                              6
HEAD BALLOONS INC                  5
Huff                               5
Holmes                             5
HAWKER SIDDELEY                    4
Hudson                             4
Hardy                            

In [450]:
df.loc[df['Make'].str.lower().str.startswith('hawk', na=False), 'Make'] = 'Hawker'

In [451]:
df.loc[df['Make'].str.lower().str.startswith('head', na=False), 'Make'] = 'Head Balloons'

In [452]:
df.loc[df['Make'].str.lower().str.startswith('helio', na=False), 'Make'] = 'Helio'

In [453]:
df.loc[df['Make'].str.lower().str.startswith('hiller', na=False), 'Make'] = 'Hiller'

In [454]:
df.loc[df['Make'].str.lower().str.startswith('howard', na=False), 'Make'] = 'Howard Aircraft'

In [455]:
df.loc[df['Make'].str.lower().str.startswith('hughes', na=False), 'Make'] = 'Hughes Helicopters'

In [456]:
df[df['Make'].str.lower().str.startswith('i', na=False)].value_counts('Make').head(50)

Make
Israel Aircraft Industries        28
I.c.a. Brasov                     27
Interstate                        17
ISRAEL AIRCRAFT INDUSTRIES         8
Intermountain Mfg. (imco)          6
IAI                                4
INFINITY                           4
INIZIATIVE INDUSTRIALI ITALIAN     3
Issoire-aviation                   3
Interavia                          3
Interplane                         3
INTERSTATE                         3
Infinity                           3
ICON AIRCRAFT INC                  3
I.C.A.-BRASOV (ROMANIA)            2
Iniziative Industriali Italian     2
Irwin                              2
Icon                               2
ICON                               2
ICP                                2
Inman                              1
Interplane Sro                     1
Ives                               1
Iverslie                           1
Ingraham                           1
Iversen                            1
Ivan Langston                    

In [457]:
df.loc[df['Make'].str.lower().str.startswith('i.c.a', na=False), 'Make'] = 'I.c.a. Brasov'

In [458]:
df.loc[df['Make'].str.lower().str.startswith('ica', na=False), 'Make'] = 'I.c.a. Brasov'

In [459]:
df.loc[df['Make'].str.lower().str.startswith('icon', na=False), 'Make'] = 'Icon'

In [460]:
df.loc[df['Make'].str.lower().str.startswith('indu', na=False), 'Make'] = 'Indus'

In [461]:
df.loc[df['Make'].str.lower().str.startswith('infini', na=False), 'Make'] = 'Infinity'

In [462]:
df.loc[df['Make'].str.lower().str.startswith('iniz', na=False), 'Make'] = 'Iniziative Industriali Italian'

In [463]:
df.loc[df['Make'].str.lower().str.startswith('interp', na=False), 'Make'] = 'Interplane'

In [464]:
df.loc[df['Make'].str.lower().str.startswith('intersta', na=False), 'Make'] = 'Interstate'

In [465]:
df[df['Make'].str.lower().str.startswith('i', na=False)].value_counts('Make').head(50)

Make
I.c.a. Brasov                     34
Israel Aircraft Industries        28
Interstate                        20
Infinity                           9
ISRAEL AIRCRAFT INDUSTRIES         8
Interplane                         7
Icon                               7
Iniziative Industriali Italian     6
Intermountain Mfg. (imco)          6
IAI                                4
Indus                              4
Interavia                          3
Issoire-aviation                   3
ICP                                2
Irwin                              2
Istenes                            1
Ireneusz                           1
Ives                               1
Iverslie                           1
Iversen                            1
Intl Ultralight                    1
Ivan Langston                      1
Irving Siewert                     1
Isaacson                           1
Iseman                             1
Iv Inc.                            1
Ismari                           

In [466]:
df.loc[df['Make'].str.lower().str.startswith('israel', na=False), 'Make'] = 'Israel Aircraft Industries'

In [467]:
df[df['Make'].str.lower().str.startswith('j', na=False)].value_counts('Make').head(50)

Make
Johnson                          20
Jones                            14
JABIRU USA SPORT AIRCRAFT LLC     9
JABIRU                            8
Just                              6
Jackson                           5
Jenkins                           5
JOHNSON                           5
Jodel                             4
Jihlavan                          4
Jonas                             4
Jetstream                         3
JONKER SAILPLANES (PTY) LTD       3
Jordan                            3
JUST AIRCRAFT LLC                 3
JDT                               2
JIHLAVAN AIRPLANES SRO            2
JOE SALOMONE                      2
Jahns                             2
Jacobs                            2
Jackman                           2
JUST AIRCRAFT                     2
Jauch                             2
J&J Ultralights                   2
Jurca                             2
Jack Mcdaniel                     2
JONES                             2
Jett                   

In [468]:
df.loc[df['Make'].str.lower().str.startswith('jabi', na=False), 'Make'] = 'Jabiru'

In [469]:
df.loc[df['Make'].str.lower().str.startswith('jihl', na=False), 'Make'] = 'Jihlavan'

In [470]:
df.loc[df['Make'].str.lower().str.startswith('jode', na=False), 'Make'] = 'Jodel'

In [471]:
df.loc[df['Make'].str.lower().str.startswith('johns', na=False), 'Make'] = 'Johnson'

In [472]:
df.loc[df['Make'].str.lower().str.startswith('jones', na=False), 'Make'] = 'Jones'

In [473]:
df.loc[df['Make'].str.lower().str.startswith('just', na=False), 'Make'] = 'Just Aircraft'

In [474]:
df[df['Make'].str.lower().str.startswith('k', na=False)].value_counts('Make').head(50)

Make
Kaman                   35
Kolb Company            17
Kitfox                  15
KOLB                     7
Kolb                     7
KUBICEK                  6
King                     6
Kelly                    4
Keuthan                  3
Kirkpatrick              3
Keith                    3
Keller                   3
Kennedy                  3
KAMAN                    3
KAMAN AEROSPACE CORP     2
Kelley                   2
KITFOX                   2
Kinner                   2
Kucklick                 2
Keesler                  2
Kauffman                 2
Kubicek                  2
Kawasaki                 2
KAWASAKI                 2
Knapp                    2
Krotje                   2
Kenney                   1
Kirner                   1
Klasing                  1
Kitty Hawk               1
Kinnson                  1
Kirby                    1
Kircher                  1
Kirchner                 1
Kenoyer                  1
Kirst Allen J            1
Kite                   

In [475]:
df.loc[df['Make'].str.lower().str.startswith('kama', na=False), 'Make'] = 'Kaman'

In [476]:
df.loc[df['Make'].str.lower().str.startswith('kawa', na=False), 'Make'] = 'Kawasaki'

In [477]:
df.loc[df['Make'].str.lower().str.startswith('kitf', na=False), 'Make'] = 'Kitfox'

In [478]:
df.loc[df['Make'].str.lower().str.startswith('kolb', na=False), 'Make'] = 'Kolb'

In [479]:
df.loc[df['Make'].str.lower().str.startswith('kubic', na=False), 'Make'] = 'Kubicek'

In [480]:
df[df['Make'].str.lower().str.startswith('l', na=False)].value_counts('Make').head(50)

Make
Luscombe                          316
Lake                              136
Lockheed                          111
Let                               111
Learjet                           109
LUSCOMBE                           95
Lancair                            32
LEARJET                            25
LET                                24
LANCAIR                            19
LAKE                               14
LOCKHEED                           11
LEARJET INC                        10
Lindstrand Balloons                 9
LINDSTRAND BALLOONS                 9
Lewis                               8
LIBERTY AEROSPACE INCORPORATED      7
LANCAIR COMPANY                     7
Lindstrand                          6
Laister                             5
Liberty Aerospace Inc.              5
Lee                                 4
Larson                              4
LINDSTRAND                          4
Long                                3
Logan                               3
Lutz   

In [481]:
df.loc[df['Make'].str.lower().str.startswith('lake', na=False), 'Make'] = 'Lake'

In [482]:
df.loc[df['Make'].str.lower().str.startswith('lanc', na=False), 'Make'] = 'Lancair'

In [483]:
df.loc[df['Make'].str.lower().str.startswith('lars', na=False), 'Make'] = 'Larson'

In [484]:
df.loc[df['Make'].str.lower().str.startswith('lear', na=False), 'Make'] = 'Learjet'

In [485]:
df.loc[df['Make'].str.lower().str.startswith('let', na=False), 'Make'] = 'Let'

In [486]:
df.loc[df['Make'].str.lower().str.startswith('liberty', na=False), 'Make'] = 'Liberty'

In [487]:
df.loc[df['Make'].str.lower().str.startswith('lindst', na=False), 'Make'] = 'Lindstrand Balloons'

In [488]:
df.loc[df['Make'].str.lower().str.startswith('lockh', na=False), 'Make'] = 'Lockheed'

In [489]:
df.loc[df['Make'].str.lower().str.startswith('long', na=False), 'Make'] = 'Long'

In [490]:
df.loc[df['Make'].str.lower().str.startswith('lusc', na=False), 'Make'] = 'Luscombe'

In [491]:
df[df['Make'].str.lower().str.startswith('m', na=False)].value_counts('Make').head(50)

Make
Mooney                           1080
Mcdonnell Douglas                 499
Maule                             443
MOONEY                            242
MAULE                             144
Mitsubishi                        126
MCDONNELL DOUGLAS                  78
Mbb                                70
MD HELICOPTER                      35
MOONEY AIRCRAFT CORP.              34
MCDONNELL DOUGLAS HELICOPTER       31
Miller                             17
MITSUBISHI                         16
Mitchell                           13
Mcculloch                          11
MCDONNELL DOUGLAS HELI CO          11
MOONEY AIRPLANE CO INC             10
Meyers Aircraft Co.                10
Maxair                              9
Monocoupe Aircraft                  9
Monnett                             9
Morrisey                            9
Murphy                              8
Martin                              8
Molino Oy                           7
Moore                               7
MEYERS 

In [492]:
df.loc[df['Make'].str.lower().str.startswith('martin', na=False), 'Make'] = 'Martin'

In [493]:
df.loc[df['Make'].str.lower().str.startswith('maul', na=False), 'Make'] = 'Maule'

In [494]:
df.loc[df['Make'].str.lower().str.startswith('MCDONNELL DOUGLAS H', na=False), 'Make'] = 'Mcdonnell Douglas Helicopters'

In [495]:
df.loc[df['Make'].str.lower().str.startswith('MCDONNELL DOUGLAS A', na=False), 'Make'] = 'Mcdonnell Douglas'

In [496]:
df[df['Make'].str.lower().str.startswith('mcdonn', na=False)].value_counts('Make').head(50)

Make
Mcdonnell Douglas                 499
MCDONNELL DOUGLAS                  78
MCDONNELL DOUGLAS HELICOPTER       31
MCDONNELL DOUGLAS HELI CO          11
MCDONNELL DOUGLAS AIRCRAFT CO       6
McDonnell Douglas                   4
Mcdonnell-douglas                   2
MCDONNELL DOUGLAS CORPORATION       1
MCDONNELL-DOUGLAS                   1
McDonnell Douglas Helicopter        1
McDonnell Douglas Helicopter C      1
McDonnell Douglas Helicopters       1
Mcdonnell Douglas Helicopter        1
Mcdonnell Douglas Helicopters       1
Name: count, dtype: int64

In [497]:
df.loc[df['Make'].str.lower().str.startswith('mcdonnell douglas h', na=False), 'Make'] = 'Mcdonnell Douglas Helicopters'

In [498]:
df[df['Make'].str.lower().str.startswith('mcdonn', na=False)].value_counts('Make').head(50)

Make
Mcdonnell Douglas                499
MCDONNELL DOUGLAS                 78
Mcdonnell Douglas Helicopters     47
MCDONNELL DOUGLAS AIRCRAFT CO      6
McDonnell Douglas                  4
Mcdonnell-douglas                  2
MCDONNELL DOUGLAS CORPORATION      1
MCDONNELL-DOUGLAS                  1
Name: count, dtype: int64

In [499]:
df.loc[df['Make'].str.lower().str.startswith('mcdonnell-douglas', na=False), 'Make'] = 'Mcdonnell Douglas'

In [500]:
df.loc[df['Make'].str.lower().str.startswith('mcdonnell douglas a', na=False), 'Make'] = 'Mcdonnell Douglas'

In [501]:
df.loc[df['Make'].str.lower().str.startswith('mcdonnell douglas c', na=False), 'Make'] = 'Mcdonnell Douglas'

In [502]:
df[df['Make'].str.lower().str.startswith('mcdonn', na=False)].value_counts('Make').head(50)

Make
Mcdonnell Douglas                509
MCDONNELL DOUGLAS                 78
Mcdonnell Douglas Helicopters     47
McDonnell Douglas                  4
Name: count, dtype: int64

In [503]:
df.loc[(df['Make'] == 'MCDONNELL DOUGLAS') | (df['Make'] == 'McDonnell Douglas'), 'Make'] = 'Mcdonnell Douglas'

In [504]:
df[df['Make'].str.lower().str.startswith('mcdonn', na=False)].value_counts('Make').head(50)

Make
Mcdonnell Douglas                591
Mcdonnell Douglas Helicopters     47
Name: count, dtype: int64

In [505]:
df[df['Make'].str.lower().str.startswith('m', na=False)].value_counts('Make').head(50)

Make
Mooney                           1080
Mcdonnell Douglas                 591
Maule                             589
MOONEY                            242
Mitsubishi                        126
Mbb                                70
Mcdonnell Douglas Helicopters      47
MD HELICOPTER                      35
MOONEY AIRCRAFT CORP.              34
Miller                             17
MITSUBISHI                         16
Martin                             16
Mitchell                           13
Mcculloch                          11
MOONEY AIRPLANE CO INC             10
Meyers Aircraft Co.                10
Monocoupe Aircraft                  9
Monnett                             9
Morrisey                            9
Maxair                              9
Murphy                              8
Molino Oy                           7
MEYERS                              7
MD HELICOPTER INC                   7
Moore                               7
Md Helicopter                       6
Morriso

In [506]:
df.loc[df['Make'].str.lower().str.startswith('maxair', na=False), 'Make'] = 'Maxair'

In [507]:
df.loc[df['Make'].str.lower().str.startswith('mbb', na=False), 'Make'] = 'MBB'

In [508]:
df.loc[df['Make'].str.lower().str.startswith('md helicopter', na=False), 'Make'] = 'Md Helicopter'

In [509]:
df.loc[df['Make'].str.lower().str.startswith('meyer', na=False), 'Make'] = 'Meyers'

In [510]:
df.loc[df['Make'].str.lower().str.startswith('miller', na=False), 'Make'] = 'Miller'

In [511]:
df.loc[df['Make'].str.lower().str.startswith('mitsub', na=False), 'Make'] = 'Mitsubishi'

In [512]:
df.loc[df['Make'].str.lower().str.startswith('monoco', na=False), 'Make'] = 'Monocoupe'

In [513]:
df.loc[df['Make'].str.lower().str.startswith('moone', na=False), 'Make'] = 'Mooney'

In [514]:
df.loc[df['Make'].str.lower().str.startswith('morris', na=False), 'Make'] = 'Morrisey'

In [515]:
df.loc[df['Make'].str.lower().str.startswith('murph', na=False), 'Make'] = 'Murphy'

In [516]:
df[df['Make'].str.lower().str.startswith('m', na=False)].value_counts('Make').head(50)

Make
Mooney                           1373
Mcdonnell Douglas                 591
Maule                             589
Mitsubishi                        142
MBB                                74
Md Helicopter                      64
Mcdonnell Douglas Helicopters      47
Meyers                             33
Miller                             33
Morrisey                           28
Monocoupe                          16
Martin                             16
Murphy                             15
Maxair                             14
Mitchell                           13
Mcculloch                          11
Monnett                             9
Moore                               7
Molino Oy                           7
Myers                               6
Mason                               5
Morgan                              5
MESSERSCHMITT-BOLKOW-BLOHM          5
Mikoyan Mig                         4
Morane-saulnier                     4
Mattison                            4
Mccoy  

In [517]:
df.loc[df['Make'].str.lower().str.startswith('messersch', na=False), 'Make'] = 'Messerschmitt'

In [518]:
df.loc[df['Make'].str.lower().str.startswith('mikoya', na=False), 'Make'] = 'Mikoyan'

In [519]:
df[df['Make'].str.lower().str.startswith('m', na=False)].value_counts('Make').head(50)

Make
Mooney                           1373
Mcdonnell Douglas                 591
Maule                             589
Mitsubishi                        142
MBB                                74
Md Helicopter                      64
Mcdonnell Douglas Helicopters      47
Meyers                             33
Miller                             33
Morrisey                           28
Monocoupe                          16
Martin                             16
Murphy                             15
Maxair                             14
Mitchell                           13
Messerschmitt                      11
Mcculloch                          11
Monnett                             9
Mikoyan                             8
Moore                               7
Molino Oy                           7
Myers                               6
Morgan                              5
Mason                               5
Mccoy                               4
Morane-saulnier                     4
Mattiso

In [520]:
df.loc[df['Make'].str.lower().str.startswith('moor', na=False), 'Make'] = 'Moore'

In [521]:
df.loc[df['Make'].str.lower().str.startswith('mong', na=False), 'Make'] = 'Mong'

In [522]:
# show make value_counts for make starting with 'm'. Value_counts less than 10
df[df['Make'].str.lower().str.startswith('m', na=False)].value_counts('Make').loc[lambda x : x<2].tail(80)

Make
Mastopietro           1
Master Donald C       1
Masselink             1
Massegee              1
Mason, Jack A.        1
                     ..
MUDRY                 1
MUDGE RAY             1
MOYNIHAN RICHARD D    1
MOYNAHAN              1
Mathew M. Kerr        1
Name: count, Length: 80, dtype: int64

In [523]:
df[df['Make'].str.lower().str.startswith('n', na=False)].value_counts('Make').head(50)

Make
North American                   294
NORTH AMERICAN                    79
Navion                            63
NAVION                            16
Naval Aircraft Factory            12
Nanchang                          11
Nord (sncan)                       9
Nihon                              8
NANCHANG CHINA                     6
NEW PIPER AIRCRAFT INC             6
Nicholson                          5
North American Rockwell Corp.      5
NORTH WING UUM INC                 5
NORTH WING                         5
North Wing                         5
NORTHWING DESIGN                   4
NORTHWING                          4
Nelson                             4
NANCHANG                           4
Northrop                           4
Noble                              3
NELSON                             3
NORTH AMERICAN/AERO CLASSICS       3
New Standard                       3
National Balloon                   3
NORD                               2
Nolan                            

In [524]:
df.loc[df['Make'].str.lower().str.startswith('nanch', na=False), 'Make'] = 'Nanchang'

In [525]:
df.loc[df['Make'].str.lower().str.startswith('navio', na=False), 'Make'] = 'Navion'

In [526]:
df.loc[df['Make'].str.lower().str.startswith('nelso', na=False), 'Make'] = 'Nelson'

In [527]:
df.loc[df['Make'].str.lower().str.startswith('new pip', na=False), 'Make'] = 'New Piper'

In [528]:
df.loc[df['Make'].str.lower().str.startswith('newel', na=False), 'Make'] = 'Newell'

In [529]:
df.loc[df['Make'].str.lower().str.startswith('nord', na=False), 'Make'] = 'Nord'

In [530]:
df.loc[df['Make'].str.lower().str.startswith('north ame', na=False), 'Make'] = 'North American'

In [531]:
df.loc[df['Make'].str.lower().str.startswith('north w', na=False), 'Make'] = 'North Wing'

In [532]:
df.loc[df['Make'].str.lower().str.startswith('northw', na=False), 'Make'] = 'North Wing'

In [533]:
df[df['Make'].str.lower().str.startswith('o', na=False)].value_counts('Make').head(50)

Make
Olson                       4
Omf                         3
O'connor                    2
Olsen                       2
Osprey                      2
O Loughlin                  1
Omac                        1
Oostdik                     1
Oo-culley                   1
Ontario Avia. Hist. Soc.    1
Oneil                       1
Olsen-gordon                1
Opus Motorsports Llc        1
Olree Robert                1
Oldfield                    1
Ohlemeier/stratton          1
Ogden Aubrey D              1
Ofria                       1
Opperman                    1
Orlando Heli Air Inc.       1
Oregon Helicopters          1
Otis                        1
Owens/richburg              1
Owen Robert R               1
Overton                     1
Oveross                     1
Ouzts Rodney                1
Otis G. Lyons               1
Otero-pitts                 1
Oestreich                   1
Osprey 2                    1
Osborne                     1
Ortmayer/parson             1
Ortiz

In [534]:
df[df['Make'].str.lower().str.startswith('p', na=False)].value_counts('Make').head(50)

Make
Piper                         11903
PIPER                          2841
Pitts                           128
Pilatus                          36
PIPER AIRCRAFT INC               27
Pzl-mielec                       22
PILATUS                          22
Pitts Special                    15
Pipistrel                        14
PITTS                            13
Pterodactyl                      13
Partenavia                       12
Pzl-bielsko                      11
Piccard                          10
PIAGGIO                           8
Porterfield                       8
Pietenpol                         8
PILATUS AIRCRAFT LTD              8
PIPER AIRCRAFT CORPORATION        8
Pzl                               7
Parsons                           7
Pioneer                           7
Pezetel                           7
Progressive Aerodyne              6
Powrachute                        5
PROGRESSIVE AERODYNE INC          5
Parker                            5
PZL-SWIDNIK            

In [535]:
df[df['Make'].str.lower().str.startswith('p', na=False)].value_counts('Make').loc[lambda x : x<2].head(60)

Make
Piper/Cub Crafters                1
Pincince                          1
Piper-aerostar                    1
Perrucci Randall J                1
Pinick                            1
Piper Pawnee                      1
Pinkham                           1
Piotrowski                        1
Pergerson                         1
Perlick                           1
Piper Cub Crafters                1
Pezold                            1
Peter A Land                      1
Pilatus Britten-norman            1
Person                            1
Pfeifler                          1
Petz                              1
Phaneuf                           1
Phantom Sport Airplane            1
Phelps                            1
Phoenix Air                       1
Piaggio Aero Industries S.p.a.    1
Piasecki Acft. Corp.              1
Pettigrew, Richard                1
Petsche Kitfox                    1
Pickel                            1
Piel Emeraude                     1
Petracca               

In [536]:
df[df['Make'].str.lower().str.startswith('p z', na=False)].value_counts('Make').head(50)

Make
P Z L -WARSZAWA-CNPSL    1
Name: count, dtype: int64

In [537]:
df.loc[df['Make'].str.lower().str.startswith('p z', na=False), 'Make'] = 'PZL'

In [538]:
df.loc[df['Make'].str.lower().str.startswith('pz', na=False), 'Make'] = 'PZL'

In [539]:
df.loc[df['Make'].str.lower().str.startswith('parke', na=False), 'Make'] = 'Parker'

In [540]:
df.loc[df['Make'].str.lower().str.startswith('partenav', na=False), 'Make'] = 'Partenavia'

In [541]:
df.loc[df['Make'].str.lower().str.startswith('pdp', na=False), 'Make'] = 'PDPS'

In [542]:
df.loc[df['Make'].str.lower().str.startswith('perth', na=False), 'Make'] = 'Perth Amboy'

In [543]:
df.loc[df['Make'].str.lower().str.startswith('phanto', na=False), 'Make'] = 'Phantom'

In [544]:
df.loc[df['Make'].str.lower().str.startswith('philli', na=False), 'Make'] = 'Phillips'

In [545]:
df.loc[df['Make'].str.lower().str.startswith('piagg', na=False), 'Make'] = 'Piaggio'

In [546]:
df.loc[df['Make'].str.lower().str.startswith('piel', na=False), 'Make'] = 'Piel'

In [547]:
df.loc[df['Make'].str.lower().str.startswith('piet', na=False), 'Make'] = 'Pietenpol'

In [548]:
df.loc[df['Make'].str.lower().str.startswith('pilat', na=False), 'Make'] = 'Pilatus'

In [549]:
df.loc[df['Make'].str.lower().str.startswith('piper', na=False), 'Make'] = 'Piper'

In [550]:
df.loc[df['Make'].str.lower().str.startswith('pipest', na=False), 'Make'] = 'Pipestrel'

In [551]:
df.loc[df['Make'].str.lower().str.startswith('pitts', na=False), 'Make'] = 'Pitts'

In [552]:
df.loc[df['Make'].str.lower().str.startswith('powr', na=False), 'Make'] = 'Powrachute'

In [553]:
df.loc[df['Make'].str.lower().str.startswith('progress', na=False), 'Make'] = 'Progressive'

In [554]:
df[df['Make'].str.lower().str.startswith('q', na=False)].value_counts('Make').head(50)

Make
QUICKSILVER                       36
Quickie                           32
Quicksilver                       29
QUAD CITY                         13
Quad City                          7
QUEST AIRCRAFT COMPANY LLC         6
Quad City Ultralight Aircraft      5
Quest                              4
Questair                           4
QUEST                              3
Questaire                          2
QUICKSILVER ENTERPRISES INC        2
Quad City Aircraft Corp            2
QUAD CITY ULTRALIGHT ACFT CORP     2
QUICKSILVER MANUFACTURING INC      2
Quick Silver                       1
Questair, Inc.                     1
Q-berry                            1
Quickie Aircraft                   1
Quickie-myers                      1
Quicksilver Aircraft               1
Quicksilver Aircraft Northeast     1
Quicksilver II                     1
Quicksilver Manufacturing          1
Quiksilver                         1
Quinn                              1
Quinn Aviation                   

In [555]:
df.loc[df['Make'].str.lower().str.startswith('quad', na=False), 'Make'] = 'Quad City'

In [556]:
df.loc[df['Make'].str.lower().str.startswith('quartz', na=False), 'Make'] = 'Quartz Mountain'

In [557]:
df.loc[df['Make'].str.lower().str.startswith('quest a', na=False), 'Make'] = 'Quest Aircraft'

In [558]:
df.loc[(df['Make'] == 'QUEST') | (df['Make'] == 'Quest'), 'Make'] = 'Quest Aircraft'

In [559]:
df[df['Make'].str.lower().str.startswith('quest', na=False)].value_counts('Make').head(50)

Make
Quest Aircraft         14
Questair                4
Questaire               2
QUESTAIR INC            1
Quest Ii Walsh Ltd.     1
Questair, Inc.          1
Name: count, dtype: int64

In [561]:
df.loc[df['Make'].str.lower().str.startswith('questa', na=False), 'Make'] = 'Questair'

In [560]:
df.loc[df['Make'].str.lower().str.startswith('quickie', na=False), 'Make'] = 'Quickie'

In [562]:
df.loc[df['Make'].str.lower().str.startswith('quick s', na=False), 'Make'] = 'Quicksilver'

In [563]:
df.loc[df['Make'].str.lower().str.startswith('quicksil', na=False), 'Make'] = 'Quicksilver'

In [564]:
df.loc[df['Make'].str.lower().str.startswith('quiks', na=False), 'Make'] = 'Quicksilver'

In [565]:
df.loc[df['Make'].str.lower().str.startswith('quinn', na=False), 'Make'] = 'Quinn'

In [567]:
df[df['Make'].str.lower().str.startswith('r', na=False)].value_counts('Make').head(50)

Make
Robinson                       940
Rockwell                       328
ROBINSON                       283
ROBINSON HELICOPTER            221
ROBINSON HELICOPTER COMPANY    179
Ryan                            92
Raven                           84
Rotorway                        56
RAYTHEON AIRCRAFT COMPANY       53
ROCKWELL INTERNATIONAL          53
RANS                            50
Rolladen-schneider              36
Republic                        28
Rans                            24
ROCKWELL                        24
Rockwell International          22
ROBINSON HELICOPTER CO          22
Raytheon Aircraft Company       17
Raytheon Corporate Jets         16
Raytheon                        16
RYAN                            16
Rutan                           16
Rotec                           15
Robinson Helicopter Company     15
RAYTHEON                        15
ROTORWAY                        12
REMOS AIRCRAFT GMBH             11
Rans, Inc.                      10
Reims Aviation 

In [571]:
df[df['Make'].str.lower().str.startswith('r', na=False)].value_counts('Make').loc[lambda x : x<2].head(60)

Make
Robert H. Low            1
Robert L. Rice           1
Robert J. Goodyear       1
Revolution Helicopter    1
Robert L. Kennedy        1
Robert J. Jackson        1
Rey                      1
Robert J. Labonte        1
Reynolds                 1
Rhea                     1
Robert Mcdonnell         1
Robert Judd              1
Richard Riley            1
Reviere                  1
Robert N. Gilmore        1
Render                   1
Roberts                  1
Renninger                1
Roberto P. Garcia        1
Robert Wood              1
Renquist                 1
Robert W. Ferrell        1
Replica                  1
Robert Van Scoyoc        1
Repliea 78 Ral           1
Resney                   1
Robert Frisbey           1
Rettberg                 1
Robert Thibodeau         1
Robert R. Reiland        1
Robert R. Callahan       1
Robert O. Ely            1
Robert Griffiths         1
Robert E. Bean           1
Robert Eldon Idler       1
Richard E. Rowland       1
Riekkenen              

In [572]:
df.loc[df['Make'].str.lower().str.startswith('rans', na=False), 'Make'] = 'Rans'

In [573]:
df.loc[df['Make'].str.lower().str.startswith('raven', na=False), 'Make'] = 'Raven'

In [574]:
df.loc[df['Make'].str.lower().str.startswith('raythe', na=False), 'Make'] = 'Raytheon'

In [575]:
df.loc[df['Make'].str.lower().str.startswith('reims', na=False), 'Make'] = 'Reims Aviation'

In [576]:
df.loc[df['Make'].str.lower().str.startswith('remos', na=False), 'Make'] = 'Remos'

In [577]:
df.loc[df['Make'].str.lower().str.startswith('republ', na=False), 'Make'] = 'Republic'

In [578]:
df.loc[df['Make'].str.lower().str.startswith('revolut', na=False), 'Make'] = 'Revolution Helicopters'

In [579]:
df.loc[df['Make'].str.lower().str.startswith('riddel', na=False), 'Make'] = 'Riddell'

In [580]:
df.loc[df['Make'].str.lower().str.startswith('robinson', na=False), 'Make'] = 'Robinson Helicopter'

In [581]:
df.loc[(df['Make'] == 'ROBIN') | (df['Make'] == 'Robin'), 'Make'] = 'Robin'

In [582]:
df[df['Make'].str.lower().str.startswith('robin', na=False)].value_counts('Make').head(50)

Make
Robinson Helicopter     1675
Robin                      8
Robin, Avions Pierre       1
Robinette                  1
Name: count, dtype: int64

In [583]:
df.loc[df['Make'].str.lower().str.startswith('rockwell', na=False), 'Make'] = 'Rockwell'

In [584]:
df.loc[df['Make'].str.lower().str.startswith('rolladen', na=False), 'Make'] = 'Rolladen-Schneider'

In [585]:
df.loc[df['Make'].str.lower().str.startswith('rose', na=False), 'Make'] = 'Rose'

In [586]:
df.loc[df['Make'].str.lower().str.startswith('rotec', na=False), 'Make'] = 'Rotec'

In [587]:
df.loc[df['Make'].str.lower().str.startswith('rotorw', na=False), 'Make'] = 'Rotorway'

In [588]:
df.loc[df['Make'].str.lower().str.startswith('rutan', na=False), 'Make'] = 'Rutan'

In [589]:
df.loc[df['Make'].str.lower().str.startswith('ryan', na=False), 'Make'] = 'Ryan'

In [593]:
df[df['Make'].str.lower().str.startswith('s', na=False)].value_counts('Make').loc[lambda x : x<3].head(60)

Make
STOL                              2
STEPHENSON                        2
STOLP STARDUSTER                  2
Snobird                           2
Snider                            2
Steinke                           2
Sorrel                            2
Scheibe                           2
Smyth Sidewinder                  2
Sloan                             2
Stampe                            2
Sawyer                            2
SWANSON                           2
Sadler                            2
Sinclair                          2
SLINGSBY                          2
Sproul                            2
Simpson                           2
Springer                          2
SKYKITS USA CORP                  2
Sprague                           2
Sportavia-putzer                  2
Sanders                           2
Skov-papworth                     2
Sport Flight                      2
Saab-scania                       2
SPORT PLANE DYNAMICS LLC          2
SILVERLIGHT AVIATION LL

In [594]:
df.loc[df['Make'].str.lower().str.startswith('saab', na=False), 'Make'] = 'Saab'

In [595]:
df.loc[df['Make'].str.lower().str.startswith('scheibe', na=False), 'Make'] = 'Scheibe'

In [596]:
df.loc[df['Make'].str.lower().str.startswith('schempp', na=False), 'Make'] = 'Schempp Hirth'

In [597]:
df.loc[df['Make'].str.lower().str.startswith('schleich', na=False), 'Make'] = 'Schleicher'

In [598]:
df.loc[df['Make'].str.lower().str.startswith('schwei', na=False), 'Make'] = 'Schweizer'

In [599]:
df.loc[df['Make'].str.lower().str.startswith('scottish', na=False), 'Make'] = 'Scottish Aviation'

In [600]:
df.loc[df['Make'].str.lower().str.startswith('short bro', na=False), 'Make'] = 'Short Brothers'

In [601]:
df.loc[df['Make'].str.lower().str.startswith('siai', na=False), 'Make'] = 'Siai Marchetti'

In [602]:
df.loc[df['Make'].str.lower().str.startswith('sikors', na=False), 'Make'] = 'Sikorsky'

In [603]:
df.loc[df['Make'].str.lower().str.startswith('silva', na=False), 'Make'] = 'Silvaire'

In [604]:
df.loc[df['Make'].str.lower().str.startswith('six ch', na=False), 'Make'] = 'Six Chuter'

In [605]:
df.loc[df['Make'].str.lower().str.startswith('skykit', na=False), 'Make'] = 'Skykits Corp'

In [606]:
df.loc[df['Make'].str.lower().str.startswith('slings', na=False), 'Make'] = 'Slingsby'

In [607]:
df[df['Make'].str.lower().str.startswith('t', na=False)].value_counts('Make').loc[lambda x : x>10].head(60)

Make
Taylorcraft             316
TAYLORCRAFT              62
Texas Helicopter         30
Thunder And Colt         23
Temco                    21
Thorp                    15
TEXTRON AVIATION INC     12
THRUSH AIRCRAFT INC      12
TECNAM                   11
Name: count, dtype: int64

In [612]:
df[df['Make'].str.lower().str.startswith('t', na=False)].value_counts('Make').loc[lambda x : x<2].head(60)

Make
Tidd Wesley                     1
Thunder & Colt Ltd              1
Textron Aviation Inc            1
Thunder Mustang                 1
Thurmond                        1
Tibert                          1
Tichacek                        1
Thomas Wild                     1
Thacker                         1
Texas Helicopter Corp.          1
Tetrault Ronad                  1
Tierra Ii                       1
Test                            1
Tessier                         1
Tesori                          1
Tesmer Glassair                 1
Tifft-v                         1
Tilbert                         1
Texas Helicopter Corporation    1
Thalman                         1
Thomas Long                     1
Tharp S Richard                 1
Thompson Howard                 1
Thomas J. Dorsey                1
Thomsen, Horst                  1
Timm                            1
Thomas E. Georges               1
Thomas D. Parkes                1
Thomas C. Piper                 1
Thornhill

In [613]:
df.loc[df['Make'].str.lower().str.startswith('taylorcr', na=False), 'Make'] = 'Taylorcraft'

In [614]:
df.loc[df['Make'].str.lower().str.startswith('tecn', na=False), 'Make'] = 'Tecnam'

In [615]:
df.loc[df['Make'].str.lower().str.startswith('temco', na=False), 'Make'] = 'Temco'

In [616]:
df.loc[df['Make'].str.lower().str.startswith('terato', na=False), 'Make'] = 'Teratorn'

In [617]:
df.loc[df['Make'].str.lower().str.startswith('texas h', na=False), 'Make'] = 'Texas Helicopter'

In [618]:
df.loc[df['Make'].str.lower().str.startswith('textro', na=False), 'Make'] = 'Textron Aviation'

In [619]:
df.loc[df['Make'].str.lower().str.startswith('thorp', na=False), 'Make'] = 'Thorp'

In [620]:
df.loc[df['Make'].str.lower().str.startswith('thrush', na=False), 'Make'] = 'Thrush Aircraft'

In [621]:
df.loc[df['Make'].str.lower().str.startswith('thunder', na=False), 'Make'] = 'Thunder And Colt'

In [622]:
df.loc[df['Make'].str.lower().str.startswith('titan', na=False), 'Make'] = 'Titan'

In [623]:
df.loc[df['Make'].str.lower().str.startswith('tl u', na=False), 'Make'] = 'TL Ultralight'

In [624]:
df.loc[df['Make'].str.lower().str.startswith('travel', na=False), 'Make'] = 'Travel Air'

In [625]:
df.loc[df['Make'].str.lower().str.startswith('trick', na=False), 'Make'] = 'Trick Trikes'

In [626]:
df.loc[df['Make'].str.lower().str.startswith('tubb', na=False), 'Make'] = 'Tubbs'

In [627]:
df.loc[df['Make'].str.lower().str.startswith('tupole', na=False), 'Make'] = 'Tupolev'

In [629]:
df.loc[df['Make'].str.lower().str.startswith('the boei', na=False), 'Make'] = 'Boeing'

In [630]:
df[df['Make'].str.lower().str.startswith('u', na=False)].value_counts('Make').loc[lambda x : x>0].head(60)

Make
Unknown                         21
ULTRAMAGIC                       9
Univair                          7
UNIVERSAL STINSON                7
ULTRAMAGIC SA                    5
Ultramagic                       5
Universal Stinson                4
Upton                            3
ULTRALIGHT AMERICA               3
UNKNOWN                          3
Ultralight Flight, Inc.          2
Ultralight Flight                2
Univar                           2
Universal                        2
Unrein                           2
Us/lta                           2
Upright                          1
Univair Aircraft Corporation     1
Ultravia Aero Int'l              1
Unander                          1
Uvify                            1
Underwood                        1
Ungerecht                        1
United Consultant Corp.          1
Utva                             1
Utterback                        1
Upchurch                         1
Uskoski                          1
Universal Globe

In [631]:
df.loc[df['Make'].str.lower().str.startswith('ultrali', na=False), 'Make'] = 'Ultralight Flight'

In [632]:
df.loc[df['Make'].str.lower().str.startswith('ultramag', na=False), 'Make'] = 'Ultramagic'

In [633]:
df.loc[df['Make'].str.lower().str.startswith('ultravia', na=False), 'Make'] = 'Ultravia Aero'

In [634]:
df.loc[df['Make'].str.lower().str.startswith('united cons', na=False), 'Make'] = 'United Consultant Corp.'

In [635]:
df.loc[df['Make'].str.lower().str.startswith('univa', na=False), 'Make'] = 'Univair'

In [636]:
df.loc[df['Make'].str.lower().str.startswith('universal s', na=False), 'Make'] = 'Universal'

In [637]:
df.loc[df['Make'].str.lower().str.startswith('unknow', na=False), 'Make'] = 'Unknown'

In [638]:
df.loc[df['Make'].str.lower().str.startswith('unregis', na=False), 'Make'] = 'Unknown'

In [639]:
df.loc[df['Make'].str.lower().str.startswith('urban a', na=False), 'Make'] = 'Urban Air'

In [640]:
df[df['Make'].str.lower().str.startswith('v', na=False)].value_counts('Make').loc[lambda x : x>0].head(60)

Make
Vans                    72
VANS                    57
Varga                   24
Vans Aircraft           24
VANS AIRCRAFT INC       11
Velocity                11
Varieze                  8
Vickers                  5
Volksplane               5
Valentin                 4
Volmer Aircraft          3
Vector                   2
VAUGHN                   2
VICKERS                  2
Viking                   2
Violet                   2
Van's Aircraft           2
Vangrunsven              2
VASHON AIRCRAFT          2
VARGA AIRCRAFT CORP.     2
Vultee                   2
Vari                     1
Vehafric                 1
Vanhoose/scorpio         1
Venus                    1
Ventura                  1
Veltman, Robert          1
Veltman                  1
Vans Aircraft Inc        1
Veith                    1
Vans Aircraft, Inc.      1
Vari-eze                 1
Vans-rupert              1
Veazie                   1
Vanwinkle                1
Vavra, Martin A.         1
Vaughn                 

In [641]:
df.loc[df['Make'].str.lower().str.startswith('valent', na=False), 'Make'] = 'Valentin'

In [642]:
df[df['Make'].str.lower().str.startswith('van', na=False)].value_counts('Make').loc[lambda x : x>0].head(60)

Make
Vans                      72
VANS                      57
Vans Aircraft             24
VANS AIRCRAFT INC         11
Vangrunsven                2
Van's Aircraft             2
Vanek                      1
Vanbenthuysen John         1
Vance Renegade             1
Vance-hamm                 1
Vandenbos/thorp            1
Vanderbeek                 1
VAN B ANTHONY              1
Vangilder                  1
Vangorden                  1
VanDenburg                 1
Vanhoose/scorpio           1
Vans Aircraft Inc          1
Vans Aircraft, Inc.        1
Vans-rupert                1
Vanatta                    1
Van's Aircraft, Inc.       1
Van's/fish                 1
VAN DUSEN JOHN M           1
VAN HOOMISSEN GERALD J     1
VAN KALKER JOHN E          1
VAN LEHN RICHARD GLENN     1
VANDERARK                  1
VANDERHOOF STEVE           1
VANEK JIM                  1
Van                        1
Van Arsdell/brent,m.       1
Van Dyke                   1
Van Grouw                  1
Van Heesw

In [644]:
df.loc[df['Make'].str.lower().str.startswith('vans', na=False), 'Make'] = 'Vans'

In [647]:
df.loc[df['Make'].str.lower().str.startswith("van's", na=False), 'Make'] = 'Vans'

In [649]:
df.loc[df['Make'].str.lower().str.startswith('varga', na=False), 'Make'] = 'Varga'

In [651]:
df.loc[df['Make'].str.lower().str.startswith('vari', na=False), 'Make'] = 'Varieze'

In [652]:
df[df['Make'].str.lower().str.startswith('vaugh', na=False)].value_counts('Make').loc[lambda x : x>0].head(60)

Make
VAUGHN              2
VAUGHAN GERALD R    1
Vaughan             1
Vaughn              1
Name: count, dtype: int64

In [653]:
df.loc[df['Make'].str.lower().str.startswith('vaugh', na=False), 'Make'] = 'Vaughn'

In [656]:
df[df['Make'].str.lower().str.startswith('w', na=False)].value_counts('Make').loc[lambda x : x<2].head(60)

Make
Welles                            1
Wente                             1
Werner                            1
William D. Weeks                  1
Wilderoder                        1
Wernega                           1
Wiles                             1
Wiley                             1
Wilford J. Tolman                 1
Wilhelm                           1
Wilkinson Daniel V                1
William Bovard                    1
Welborn                           1
Willbird                          1
William C. Keiling                1
Wesley                            1
Willey                            1
Willi/bensen                      1
Wells/rand                        1
William A. Waas                   1
William Allen                     1
Wilde Mickey                      1
Wilbur D. Batman                  1
Wessel                            1
Wild Goose                        1
Wheeler Acft. Co.                 1
Weyerts                           1
Weston                 

In [657]:
df.loc[df['Make'].str.lower().str.startswith('waco', na=False), 'Make'] = 'Waco'

In [658]:
df.loc[df['Make'].str.lower().str.startswith('weatherl', na=False), 'Make'] = 'Weatherly'

In [659]:
df.loc[df['Make'].str.lower().str.startswith('weber', na=False), 'Make'] = 'Weber'

In [660]:
df.loc[df['Make'].str.lower().str.startswith('westland', na=False), 'Make'] = 'Westland Helicopters'

In [661]:
df.loc[df['Make'].str.lower().str.startswith('wheele', na=False), 'Make'] = 'Wheeler'

In [662]:
df.loc[df['Make'].str.lower().str.startswith('white', na=False), 'Make'] = 'White'

In [663]:
df.loc[df['Make'].str.lower().str.startswith('whittman', na=False), 'Make'] = 'Whittman'

In [664]:
df.loc[df['Make'].str.lower().str.startswith('williams hel', na=False), 'Make'] = 'Williams Helicopter'

In [665]:
df.loc[df['Make'].str.lower().str.startswith('wsk', na=False), 'Make'] = 'WSK'

In [669]:
df[df['Make'].str.lower().str.startswith('x', na=False)].value_counts('Make').loc[lambda x : x<11].head(60)

Make
XTREMEAIR GMBH    3
X-AIR LLC         1
Xtremeair GMBH    1
Name: count, dtype: int64

In [670]:
df.loc[df['Make'].str.lower().str.startswith('xtre', na=False), 'Make'] = 'Xtremeair'

In [675]:
df[df['Make'].str.lower().str.startswith('y', na=False)].value_counts('Make').loc[lambda x : x>0].head(60)

Make
Yakovlev               31
YAKOVLEV               12
Young                  12
Yonchak                 1
Yasecko                 1
Yates                   1
Yeager Inc.             1
Yoakley                 1
Yoder                   1
York                    1
Yaple, Robert G.        1
Yothment                1
Youkey                  1
Young-losey             1
Young-owens             1
Younkin J R             1
Yarnell                 1
Yamokoski               1
YAKOVLEV/CHINNERY       1
Yadon                   1
YUNEEC                  1
YOUNGS WILLIAM D        1
YOUNG ROBERT HERMAN     1
YOUNG JOHN E            1
YOUNG DEE C             1
YOST EDWIN S            1
YORK RONALD J           1
YORK                    1
YATES W/LAZAR M         1
YATES MIKE E            1
YAMOKOSKI WILLIAM       1
YAKOVLEV/DAY            1
Yowell                  1
Name: count, dtype: int64

In [676]:
df.loc[df['Make'].str.lower().str.startswith('yakov', na=False), 'Make'] = 'Yakovlev'

In [677]:
df.loc[df['Make'].str.lower().str.startswith('yamok', na=False), 'Make'] = 'Yamokoski'

In [678]:
df[df['Make'].str.lower().str.startswith('z', na=False)].value_counts('Make').loc[lambda x : x>0].head(60)

Make
ZENITH                      37
Zenair                      27
Zenith                       7
Zimmerman                    4
ZENAIR LTD                   4
ZIVKO AERONAUTICS INC        4
ZLIN                         4
ZENAIR                       3
Zengel                       2
Zlin                         2
Z-HI-MAX                     1
Zhuhai Helicopter Co.        1
Zidek                        1
Zielke                       1
Ziermann                     1
Zilz                         1
Zivko Aeronautics Inc.       1
Zito                         1
Zivko                        1
Zivko Aeronautics            1
Zlin Aviation                1
Zlin Aviation S.r.o.         1
Zodiac                       1
Zorn                         1
Zukowski                     1
Zenair Zodiac                1
Zeidman                      1
Zemp                         1
Zeigler                      1
ZEILER                       1
ZENITH ACFT CO               1
ZENITH AIRCRAFT CO           1
ZIM

In [679]:
df.loc[df['Make'].str.lower().str.startswith('zenai', na=False), 'Make'] = 'Zenair'

In [680]:
df.loc[df['Make'].str.lower().str.startswith('zeni', na=False), 'Make'] = 'Zenith'

In [681]:
df.loc[df['Make'].str.lower().str.startswith('zimmerm', na=False), 'Make'] = 'Zimmerman'

In [682]:
df.loc[df['Make'].str.lower().str.startswith('zivk', na=False), 'Make'] = 'Zivko Aeronautics'

In [683]:
df.loc[df['Make'].str.lower().str.startswith('zli', na=False), 'Make'] = 'Zlin'

In [722]:
df['Make'].value_counts().loc[lambda x : x == 3].tail(60)

Make
S.O.C.A.T.A.                      3
Keller                            3
Mueller                           3
Schlitter                         3
Allied Ag Cat Productions, Inc    3
MX AIRCRAFT LLC                   3
Teague                            3
National Balloon                  3
Downer                            3
Noble                             3
Mcilraith                         3
Root                              3
Smith Aerostar                    3
Acro Sport                        3
Barr                              3
Kennedy                           3
Henderson                         3
Spartan                           3
Ferguson                          3
Striplin                          3
Stephens                          3
Ted Smith                         3
AEROPRO CZ S R O                  3
Mcclung                           3
SOLO WINGS                        3
Wallace                           3
Jordan                            3
SONEX AIRCRAFT         

In [723]:
df.loc[df['Make'].str.lower().str.startswith('aero vodo', na=False), 'Make'] = 'Aero Vodochody'

In [724]:
df.loc[df['Make'].str.lower().str.startswith('aeromot', na=False), 'Make'] = 'Aeromot'

In [725]:
df.loc[df['Make'].str.lower().str.startswith('aeropro', na=False), 'Make'] = 'Aeropro CZ'

In [726]:
df.loc[df['Make'].str.lower().str.startswith('aerostar', na=False), 'Make'] = 'Aerostar'

In [727]:
df.loc[df['Make'].str.lower().str.startswith('aerotek', na=False), 'Make'] = 'Aerotek'

In [728]:
df.loc[df['Make'].str.lower().str.startswith('air cre', na=False), 'Make'] = 'Air Creation'

In [729]:
df.loc[df['Make'].str.lower().str.startswith('aircraft mfg', na=False), 'Make'] = 'Aircraft Mfg and Dev'

In [730]:
df.loc[df['Make'].str.lower().str.startswith('alon', na=False), 'Make'] = 'Alon'

In [731]:
df.loc[df['Make'].str.lower().str.startswith('amateur b', na=False), 'Make'] = 'Amateur Built'

In [732]:
df.loc[df['Make'].str.lower().str.startswith('atr', na=False), 'Make'] = 'ATR'

In [733]:
df.loc[df['Make'].str.lower().str.startswith('autogyr', na=False), 'Make'] = 'AutoGyro'

In [734]:
df.loc[df['Make'].str.lower().str.startswith('avid', na=False), 'Make'] = 'Avid'

In [735]:
df.loc[df['Make'].str.lower().str.startswith('balloon w', na=False), 'Make'] = 'Balloon Works'

In [736]:
df.loc[df['Make'].str.lower().str.startswith('brantl', na=False), 'Make'] = 'Brantly'

In [737]:
df.loc[df['Make'].str.lower().str.startswith('british ae', na=False), 'Make'] = 'British Aerospace'

In [738]:
df.loc[df['Make'].str.lower().str.startswith('britten', na=False), 'Make'] = 'Britten Norman'

In [739]:
df.loc[df['Make'].str.lower().str.startswith('buckeye', na=False), 'Make'] = 'Buckeye'

In [740]:
df.loc[df['Make'].str.lower().str.startswith('burkhart', na=False), 'Make'] = 'Burkhart Grob'

In [741]:
df.loc[df['Make'].str.lower().str.startswith('canadair', na=False), 'Make'] = 'Canadair'

In [742]:
df.loc[df['Make'].str.lower().str.startswith('cassutt', na=False), 'Make'] = 'Cassutt'

In [743]:
df.loc[df['Make'].str.lower().str.startswith('cgs', na=False), 'Make'] = 'CGS Aviation'

In [744]:
df.loc[df['Make'].str.lower().str.startswith('classic airc', na=False), 'Make'] = 'Classic Aircraft Corp'

In [745]:
df.loc[df['Make'].str.lower().str.startswith('continental', na=False), 'Make'] = 'Continental Copters'

In [746]:
df.loc[df['Make'].str.lower().str.startswith('convair', na=False), 'Make'] = 'Convair'

In [747]:
df.loc[df['Make'].str.lower().str.startswith('cosmos', na=False), 'Make'] = 'Cosmos'

In [748]:
df.loc[df['Make'].str.lower().str.startswith('costruzioni', na=False), 'Make'] = 'Costruzioni Aeronautiche Tecna'

In [749]:
df.loc[df['Make'].str.lower().str.startswith('curtis', na=False), 'Make'] = 'Curtiss-Wright'

In [750]:
df.loc[df['Make'].str.lower().str.startswith('czech a', na=False), 'Make'] = 'Czech Aircraft Works'

In [751]:
df.loc[df['Make'].str.lower().str.startswith('czech s', na=False), 'Make'] = 'Czech Sport Aircraft'

In [752]:
df.loc[df['Make'].str.lower().str.startswith('downer', na=False), 'Make'] = 'Downer Aircraft Industries'

In [753]:
df.loc[df['Make'].str.lower().str.startswith('pipistrel', na=False), 'Make'] = 'Pipistrel'

In [754]:
df.loc[df['Make'].str.lower().str.startswith('socata', na=False), 'Make'] = 'Socata'

In [755]:
df.loc[df['Make'].str.lower().str.startswith('sonex', na=False), 'Make'] = 'Sonex'

In [756]:
df.loc[df['Make'].str.lower().str.startswith('stearm', na=False), 'Make'] = 'Stearman Aircraft'

In [757]:
df.loc[df['Make'].str.lower().str.startswith('steen', na=False), 'Make'] = 'Steen'

In [758]:
df.loc[df['Make'].str.lower().str.startswith('stemme', na=False), 'Make'] = 'Stemme'

In [759]:
df.loc[df['Make'].str.lower().str.startswith('stinson', na=False), 'Make'] = 'Stinson'

In [760]:
df.loc[df['Make'].str.lower().str.startswith('sukh', na=False), 'Make'] = 'Sukhoi'

In [761]:
df.loc[df['Make'].str.lower().str.startswith('swearing', na=False), 'Make'] = 'Swearingen'

In [763]:
df['Make'].value_counts().loc[lambda x : x>50].head(60)

Make
Cessna                 26903
Piper                  14808
Beech                   5372
Bell                    2792
Boeing                  2728
Robinson Helicopter     1675
Grumman                 1632
Mooney                  1373
Bellanca                1037
Hughes Helicopters       941
Air Tractor              911
Schweizer                802
Aeronca                  633
Mcdonnell Douglas        591
Maule                    589
Champion                 517
De Havilland             512
Airbus                   506
Cirrus Design            467
Rockwell                 437
Stinson                  433
Aero Commander           425
Luscombe                 412
Aerospatiale             395
North American           392
Taylorcraft              389
Hiller                   361
Enstrom                  303
Eurocopter               295
Douglas                  284
Ayres                    284
Aviat                    259
Embraer                  257
Ercoupe                  247
Sikorsky 

In [1181]:
# Show category value_counts for particular make
df[df['Make'] == 'Textron Aviation'].value_counts('Aircraft_Category', dropna=False)

C:\Users\Joseph Harvey\anaconda3\Lib\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


Aircraft_Category
Airplane    17
Name: count, dtype: int64

In [1179]:
# Fill in Aircraft_Category as Airplane for particular make
df.loc[df['Make'] == 'Steen', 'Aircraft_Category'] = 'Airplane'

In [1164]:
# Fill in Aircraft_Category as Helicopter for particular make
df.loc[df['Make'] == 'American Eurocopter', 'Aircraft_Category'] = 'Helicopter'

In [1142]:
# Fill in Aircraft_Category as Balloon for particular make
df.loc[df['Make'] == 'Ultramagic', 'Aircraft_Category'] = 'Balloon'

In [1140]:
# Fill in Aircraft_Category as Glider for particular make
df.loc[df['Make'] == 'Eiriavion Oy', 'Aircraft_Category'] = 'Glider'

In [1014]:
# Fill in Aircraft_Category as Gyrocraft for particular make
df.loc[df['Make'] == 'Benson', 'Aircraft_Category'] = 'Gyrocraft'

In [1119]:
# Fill in Aircraft_Category as Weight-Shift for particular make
df.loc[df['Make'] == 'Evolution', 'Aircraft_Category'] = 'Weight-Shift'

In [1076]:
# Fill in Aircraft_Category as Powered Parachute for particular make
df.loc[df['Make'] == 'Buckeye', 'Aircraft_Category'] = 'Powered Parachute'

In [1182]:
# Export df as a separate file
df.to_csv('working-df/cleaned_aviation_data_category2.csv', index=False)